In [1]:
from deep_q_network import DQNAgent
from base_rl_agent_torch import ReplayMemory
from base_neural_model import EstimatorModelBase
import utils
import random
import copy
from Data import *

In [2]:
# Fix seeds for reproducibility.
#utils.fix_seeds()

In [3]:
env = Data(unknown_rate=0.90)
env.loadfile("heart.csv")
env.normalize()
env.cluster_K_means(7)
# makes the costs uniform
env.set_costs()
# tau relaxes the terminating condition a little
env.tau = 1/13
# max_cost is the fixed maximum cost the agent can take
env.max_cost = 4/13 
# partition into training/test sets
test_env = env.split(0.80)

[0.07260726072607261, 0.1419141914191419, 0.21782178217821782, 0.22112211221122113, 0.036303630363036306, 0.2145214521452145, 0.09570957095709572]


In [4]:
MAX_EPISODES = 12000
MAX_STEPS = 13
BATCH_SIZE = 32
buffer = ReplayMemory(10000)

# Initiate the agent
model = EstimatorModelBase
agent = DQNAgent(env, model, buffer,max_steps=100, max_episodes=MAX_EPISODES,epsilon_decay=0.9996)

Using cpu


In [5]:
#Train
episode_scores = agent.train(env, MAX_EPISODES, MAX_STEPS, BATCH_SIZE)

Episode 0: -0.24177926527696805 	 1
Episode 1: -0.06784700975411362 	 0.9996
Episode 2: -0.7653327660310841 	 0.9992001600000001
Episode 3: 0.0475376056305018 	 0.9988004799360002
Episode 4: -0.18835164835164842 	 0.9984009597440259
Episode 5: 0.07761495666140483 	 0.9980015993601283
Episode 6: -0.39406593406593404 	 0.9976023987203844
Episode 7: -0.18218592196896743 	 0.9972033577608963
Episode 8: -0.14784700975411358 	 0.996804476417792
Episode 9: -0.3169142037230337 	 0.9964057546272249
Episode 10: -0.2369142037230337 	 0.996007192325374
Episode 11: 0.017362637362637434 	 0.9956087894484439
Episode 12: 0.07761495666140483 	 0.9952105459326646
Episode 13: -0.06263736263736264 	 0.9948124617142915
Episode 14: 0.22307692307692317 	 0.9944145367296058
Episode 15: -0.10218592196896747 	 0.994016770914914
Episode 16: -0.43877094258515037 	 0.9936191642065481
Episode 17: 0.017362637362637434 	 0.9932217165408654
Episode 18: -0.06784700975411362 	 0.9928244278542492
Episode 19: -0.147847009

Episode 161: -0.34029485035696205 	 0.9376177927449635
Episode 162: 0.22307692307692317 	 0.9372427456278656
Episode 163: -0.3766781289643079 	 0.9368678485296145
Episode 164: -0.5392001766533306 	 0.9364931013902027
Episode 165: -0.3393658407906297 	 0.9361185041496467
Episode 166: -0.21323067162296025 	 0.9357440567479869
Episode 167: -0.3766781289643079 	 0.9353697591252877
Episode 168: -0.14784700975411358 	 0.9349956112216377
Episode 169: -0.3608181097241878 	 0.9346216129771491
Episode 170: -0.06784700975411362 	 0.9342477643319582
Episode 171: 0.3384615384615386 	 0.9338740652262255
Episode 172: -0.09792436078501651 	 0.933500515600135
Episode 173: -0.09784605623834483 	 0.933127115393895
Episode 174: -0.47406593406593406 	 0.9327538645477376
Episode 175: -0.15296703296703293 	 0.9323807630019185
Episode 176: -0.10218592196896747 	 0.9320078106967178
Episode 177: -0.34029485035696205 	 0.9316350075724391
Episode 178: -0.6402597392405075 	 0.9312623535694102
Episode 179: -0.34129

Episode 315: -0.5643956043956044 	 0.8815926244431377
Episode 316: -0.21323067162296025 	 0.8812399873933605
Episode 317: -0.18835164835164842 	 0.8808874913984032
Episode 318: 0.3384615384615386 	 0.8805351364018439
Episode 319: -0.2369142037230337 	 0.8801829223472832
Episode 320: 0.07761495666140483 	 0.8798308491783443
Episode 321: -0.6797802197802199 	 0.879478916838673
Episode 322: -0.14784700975411358 	 0.8791271252719375
Episode 323: 0.07761495666140483 	 0.8787754744218288
Episode 324: -0.28081810972418786 	 0.8784239642320602
Episode 325: -0.3608181097241878 	 0.8780725946463673
Episode 326: -0.5842329090006687 	 0.8777213656085088
Episode 327: -0.017846056238344787 	 0.8773702770622654
Episode 328: 0.22307692307692317 	 0.8770193289514405
Episode 329: -0.32119191805636416 	 0.8766685212198599
Episode 330: -0.017846056238344787 	 0.8763178538113721
Episode 331: -0.3591495455170606 	 0.8759673266698476
Episode 332: -0.10218592196896747 	 0.8756169397391796
Episode 333: -0.5986

Episode 470: -0.12152958833841829 	 0.8285835435950066
Episode 471: 0.017362637362637434 	 0.8282521101775686
Episode 472: -0.3169142037230337 	 0.8279208093334977
Episode 473: -0.1332306716229602 	 0.8275896410097643
Episode 474: -0.25936584079062974 	 0.8272586051533605
Episode 475: -0.21330897616963193 	 0.8269277017112991
Episode 476: -0.0288705933236324 	 0.8265969306306147
Episode 477: 0.07761495666140483 	 0.8262662918583624
Episode 478: 0.07761495666140483 	 0.8259357853416192
Episode 479: -0.41097708865702876 	 0.8256054110274825
Episode 480: 0.013198693415647944 	 0.8252751688630716
Episode 481: -0.41097708865702876 	 0.8249450587955264
Episode 482: -0.32119191805636416 	 0.8246150807720082
Episode 483: -0.24177926527696805 	 0.8242852347396994
Episode 484: -0.12152958833841829 	 0.8239555206458036
Episode 485: -0.2369142037230337 	 0.8236259384375453
Episode 486: -0.1332306716229602 	 0.8232964880621703
Episode 487: -0.4011919180563642 	 0.8229671694669455
Episode 488: -0.26

Episode 621: -0.14784700975411358 	 0.7800090995693889
Episode 622: -0.4274487670148579 	 0.7796970959295612
Episode 623: -0.4274487670148579 	 0.7793852170911894
Episode 624: 0.07761495666140483 	 0.7790734630043529
Episode 625: -0.2369142037230337 	 0.7787618336191512
Episode 626: -0.1332306716229602 	 0.7784503288857035
Episode 627: 0.22307692307692317 	 0.7781389487541492
Episode 628: -0.34029485035696205 	 0.7778276931746476
Episode 629: 0.07761495666140483 	 0.7775165620973777
Episode 630: -0.18835164835164842 	 0.7772055554725388
Episode 631: 0.017362637362637434 	 0.7768946732503498
Episode 632: -0.3608181097241878 	 0.7765839153810496
Episode 633: -0.21323067162296025 	 0.7762732818148973
Episode 634: -0.06263736263736264 	 0.7759627725021714
Episode 635: -0.49097708865702877 	 0.7756523873931706
Episode 636: -0.1332306716229602 	 0.7753421264382133
Episode 637: -0.5392001766533306 	 0.7750319895876381
Episode 638: -0.24543349433957237 	 0.7747219767918031
Episode 639: 0.07761

...

Episode 11199: -0.17792436078501653 	 0.01132779137468401
Episode 11200: 0.22307692307692317 	 0.011323260258134137
Episode 11201: 0.07761495666140483 	 0.011318730954030885
Episode 11202: 0.22307692307692317 	 0.011314203461649273
Episode 11203: 0.19299957204602025 	 0.011309677780264614
Episode 11204: -0.2933089761696319 	 0.011305153909152509
Episode 11205: 0.22307692307692317 	 0.011300631847588848
Episode 11206: 0.07761495666140483 	 0.011296111594849813
Episode 11207: 0.07761495666140483 	 0.011291593150211873
Episode 11208: 0.07761495666140483 	 0.011287076512951789
Episode 11209: -0.25936584079062974 	 0.011282561682346608
Episode 11210: -0.16177926527696807 	 0.01127804865767367
Episode 11211: 0.0475376056305018 	 0.0112735374382106
Episode 11212: -0.06263736263736264 	 0.011269028023235317
Episode 11213: -0.24177926527696805 	 0.011264520412026023
Episode 11214: -0.2683516483516484 	 0.011260014603861213
Episode 11215: 0.22307692307692317 	 0.01125551059801967
Episode 11216: 

Episode 11349: -0.42029485035696207 	 0.010667984109793414
Episode 11350: -0.1332306716229602 	 0.010663716916149497
Episode 11351: -0.06784700975411362 	 0.010659451429383038
Episode 11352: 0.07761495666140483 	 0.010655187648811286
Episode 11353: -0.1332306716229602 	 0.010650925573751762
Episode 11354: 0.017362637362637434 	 0.010646665203522261
Episode 11355: -0.21330897616963193 	 0.010642406537440852
Episode 11356: 0.07761495666140483 	 0.010638149574825876
Episode 11357: 0.22307692307692317 	 0.010633894314995946
Episode 11358: 0.07761495666140483 	 0.010629640757269947
Episode 11359: 0.017362637362637434 	 0.01062538890096704
Episode 11360: 0.07761495666140483 	 0.010621138745406654
Episode 11361: -0.21330897616963193 	 0.010616890289908492
Episode 11362: -0.18835164835164842 	 0.010612643533792529
Episode 11363: 0.07761495666140483 	 0.010608398476379012
Episode 11364: -0.06784700975411362 	 0.010604155116988461
Episode 11365: -0.21323067162296025 	 0.010599913454941667
Episod

Episode 11496: 0.22307692307692317 	 0.010058674068621997
Episode 11497: -0.06784700975411362 	 0.010054650598994548
Episode 11498: 0.22307692307692317 	 0.01005062873875495
Episode 11499: -0.21330897616963193 	 0.010046608487259448
Episode 11500: -0.10218592196896747 	 0.010042589843864545
Episode 11501: -0.06680130658435202 	 0.010038572807927
Episode 11502: -0.14784700975411358 	 0.01003455737880383
Episode 11503: -0.16177926527696807 	 0.010030543555852308
Episode 11504: 0.07761495666140483 	 0.010026531338429968
Episode 11505: -0.5392001766533306 	 0.010022520725894596
Episode 11506: 0.22307692307692317 	 0.010018511717604239
Episode 11507: -0.18835164835164842 	 0.010014504312917198
Episode 11508: -0.18835164835164842 	 0.010010498511192031
Episode 11509: 0.22307692307692317 	 0.010006494311787555
Episode 11510: -0.0288705933236324 	 0.01000249171406284
Episode 11511: -0.18835164835164842 	 0.01
Episode 11512: 0.07761495666140483 	 0.01
Episode 11513: -0.10218592196896747 	 0.01


Episode 11684: -0.16177926527696807 	 0.01
Episode 11685: 0.22307692307692317 	 0.01
Episode 11686: 0.017362637362637434 	 0.01
Episode 11687: 0.22307692307692317 	 0.01
Episode 11688: 0.07761495666140483 	 0.01
Episode 11689: -0.381393655464049 	 0.01
Episode 11690: 0.07761495666140483 	 0.01
Episode 11691: -0.16177926527696807 	 0.01
Episode 11692: 0.22307692307692317 	 0.01
Episode 11693: 0.13274725274725285 	 0.01
Episode 11694: 0.07761495666140483 	 0.01
Episode 11695: -0.1439812254060143 	 0.01
Episode 11696: -0.0288705933236324 	 0.01
Episode 11697: 0.017362637362637434 	 0.01
Episode 11698: -0.17792436078501653 	 0.01
Episode 11699: 0.07761495666140483 	 0.01
Episode 11700: 0.07761495666140483 	 0.01
Episode 11701: 0.22307692307692317 	 0.01
Episode 11702: -0.10218592196896747 	 0.01
Episode 11703: 0.017362637362637434 	 0.01
Episode 11704: 0.07761495666140483 	 0.01
Episode 11705: -0.10887059332363236 	 0.01
Episode 11706: -0.06784700975411362 	 0.01
Episode 11707: -0.21330897

Episode 11885: -0.06784700975411362 	 0.01
Episode 11886: 0.07761495666140483 	 0.01
Episode 11887: -0.25936584079062974 	 0.01
Episode 11888: -0.06680130658435202 	 0.01
Episode 11889: -0.16177926527696807 	 0.01
Episode 11890: -0.06784700975411362 	 0.01
Episode 11891: -0.0288705933236324 	 0.01
Episode 11892: 0.017362637362637434 	 0.01
Episode 11893: 0.22307692307692317 	 0.01
Episode 11894: -0.24177926527696805 	 0.01
Episode 11895: -0.10887059332363236 	 0.01
Episode 11896: 0.017362637362637434 	 0.01
Episode 11897: -0.32119191805636416 	 0.01
Episode 11898: 0.22307692307692317 	 0.01
Episode 11899: 0.22307692307692317 	 0.01
Episode 11900: 0.07761495666140483 	 0.01
Episode 11901: -0.18835164835164842 	 0.01
Episode 11902: -0.06784700975411362 	 0.01
Episode 11903: 0.13274725274725285 	 0.01
Episode 11904: -0.3169142037230337 	 0.01
Episode 11905: 0.07761495666140483 	 0.01
Episode 11906: 0.3384615384615386 	 0.01
Episode 11907: 0.07761495666140483 	 0.01
Episode 11908: 0.223076

In [7]:

for iterations in range(30):
    # Now we evaluate the trained model by taking greedy actions
    total1 = 0
    total2 = 0

    N=len(test_env.data)
    test_env.it = 0
    for it in range(N):
        observation = test_env.next_element() # Current observed state
        ob_cp = (copy.copy(observation[0]),copy.copy(observation[1]))
        #print("ob #:",observation[0])
        init_s = test_env.score(observation)[0]
        total_reward = 0
        agent.epsilon = 0
        i= 0
        done = False

        while True:
            if done:
                break;
            action = agent.get_action(observation,env)
            observation, reward, done, info = test_env.step(observation, action)
            total_reward += reward

        total1 += (test_env.score(observation)[0])

        observation = ob_cp
        done = False
        init_s = test_env.score(observation)[0]
        while True:
            if done:
                break;
            actions = test_env.actions(observation)
            if len(actions) == 0:
                break
            r = random.randint(0,len(actions)-1)
            action = actions[0]

            observation, reward, done, info = test_env.step(observation, action)
        total2+=(test_env.score(observation)[0])

    print("total final score RL:", total1)
    print("total final rand:", total2)
    print()

# Saving the network
#agent.to_pickle("somefile.pickle")


total final score RL: 32.104627854397535
total final rand: 39.19387311394151

total final score RL: 31.168099790419728
total final rand: 39.27734624529568

total final score RL: 29.261351675832213
total final rand: 38.13216504242726

total final score RL: 30.625081289763102
total final rand: 40.05041818341723

total final score RL: 29.38838632219919
total final rand: 38.91096282295838

total final score RL: 28.78925729301363
total final rand: 40.3384300009611

total final score RL: 32.396785492315196
total final rand: 39.8614591350818

total final score RL: 31.654644932033133
total final rand: 38.63961368937754

total final score RL: 31.286389813648697
total final rand: 39.65719537220505

total final score RL: 32.99882705981958
total final rand: 38.68060477427206

total final score RL: 30.375786866682247
total final rand: 40.55805643379777

total final score RL: 31.12018024223631
total final rand: 39.77131955649229

total final score RL: 36.09875583949075
total final rand: 40.954899121